In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("NotebookSpark") \
                            .master("spark://spark-master:7077") \
                            .getOrCreate()
print(1+1)
df = spark.read.option("mode","DROPMALFORMED").csv("data/orders.csv")
df.printSchema()
df.show()
spark.stop()

2
root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)

+--------+-----------+----------+------------+--------+
|     _c0|        _c1|       _c2|         _c3|     _c4|
+--------+-----------+----------+------------+--------+
|order_id|customer_id|order_date|total_amount|currency|
|    1001|          1|2025-06-01|      500000|     VND|
|    1002|          2|2025-06-02|      750000|     VND|
|    1003|          3|2025-06-01|         100|     USD|
|    1004|          4|2025-06-03|          80|     GBP|
|    1005|          1|2025-06-05|          50|     EUR|
+--------+-----------+----------+------------+--------+



In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
   .appName("Test Iceberg Write") \
   .enableHiveSupport() \
   .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog") \
   .config("spark.sql.catalog.spark_catalog.type", "hive") \
   .config("spark.sql.catalog.spark_catalog.uri", "thrift://hive-metastore:9083") \
   .config("spark.jars", "/home/jovyan/jars/iceberg-spark-runtime-3.5_2.12-1.9.2.jar") \
   .getOrCreate()
# Tạo DataFrame đơn giản
print("Today is ", 1+1)
df = spark.createDataFrame([
   (1, "Alice"),
   (2, "Bob"),
   (3, "NLHT")
], ["id", "name"])
# Ghi vào bảng Iceberg (Hive catalog)
spark.table("hive_catalog.default.peoples").show(truncate=False)
#df.writeTo("spark_catalog.default.peoples").append()
df.show()

Today is  2


AnalysisException: [REQUIRES_SINGLE_PART_NAMESPACE] spark_catalog requires a single-part namespace, but got `hive_catalog`.`default`.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
def main():
   # Khởi SparkSession (nạp spark-defaults.conf và hive-site.xml)
   spark = SparkSession.builder \
       .appName("IcebergHiveCatalogExample") \
       .master("spark://spark-master:7077") \
       .enableHiveSupport() \
       .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog") \
       .config("spark.sql.catalog.spark_catalog.type", "hive") \
       .config("spark.sql.catalog.spark_catalog.uri", "thrift://hive-metastore:9083") \
       .config("spark.jars", "/home/jovyan/jars/iceberg-spark-runtime-3.5_2.12-1.9.2.jar") \
       .getOrCreate()
   
   # Tạo DataFrame mẫu
   schema = StructType([
       StructField("id",   IntegerType(), False),
       StructField("name", StringType(),  False),
       StructField("age",  IntegerType(), True)
   ])
   data = [
       (1, "Nguyễn Văn A", 30),
       (2, "Trần Thị B",   25),
       (3, "Lê Văn C",     28),
       (4, "Nguyễn Lương Hoàng Tùng", 21)
   ]
   df = spark.createDataFrame(data, schema)
   
   print("=== Dữ liệu mẫu ===")
   df.show(truncate=False)
   
   # Tạo namespace (database) nếu chưa có
   spark.sql("CREATE NAMESPACE IF NOT EXISTS hive_catalog.default")
   
   # Tạo table Iceberg (nếu chưa tồn tại)
   spark.sql("""
     CREATE TABLE IF NOT EXISTS hive_catalog.default.users (
       id   INT,
       name STRING,
       age  INT
     ) USING iceberg
   """)
   
   # Ghi dữ liệu vào bảng (append)
   df.writeTo("hive_catalog.default.users").append()
   print(">>> Đã ghi dữ liệu vào hive_catalog.default.users")
   
   # Đọc lại và hiển thị
   spark.table("hive_catalog.default.users").show(truncate=False)
   print("=== Dữ liệu trong bảng Iceberg ===")
   
   # Hiển thị lịch sử commit, sắp xếp theo made_current_at DESC
   hist_df = spark.table("hive_catalog.default.users.history") \
                 .orderBy(col("made_current_at").desc())
   hist_df.show(10, truncate=False)
   print("=== Lịch sử commit của bảng (mới nhất trước) ===")
   
   # Xem danh sách snapshots
   snap_df = spark.table("hive_catalog.default.users.snapshots")
   snap_df.show(truncate=True)
   print("=== Danh sách snapshot hiện tại ===")
   
   spark.stop()

if __name__ == "__main__":
   main()

=== Dữ liệu mẫu ===
+---+-----------------------+---+
|id |name                   |age|
+---+-----------------------+---+
|1  |Nguyễn Văn A           |30 |
|2  |Trần Thị B             |25 |
|3  |Lê Văn C               |28 |
|4  |Nguyễn Lương Hoàng Tùng|21 |
+---+-----------------------+---+



Py4JJavaError: An error occurred while calling o36.sql.
: org.apache.spark.SparkException: [INTERNAL_ERROR] Undefined error message parameter for error class: '_LEGACY_ERROR_TEMP_1055'. Parameters: Map(database -> hive_catalog.default)
	at org.apache.spark.SparkException$.internalError(SparkException.scala:77)
	at org.apache.spark.SparkException$.internalError(SparkException.scala:81)
	at org.apache.spark.ErrorClassesJsonReader.getErrorMessage(ErrorClassesJSONReader.scala:56)
	at org.apache.spark.SparkThrowableHelper$.getMessage(SparkThrowableHelper.scala:55)
	at org.apache.spark.SparkThrowableHelper$.getMessage(SparkThrowableHelper.scala:42)
	at org.apache.spark.sql.AnalysisException.<init>(AnalysisException.scala:50)
	at org.apache.spark.sql.AnalysisException.<init>(AnalysisException.scala:73)
	at org.apache.spark.sql.errors.QueryCompilationErrors$.invalidDatabaseNameError(QueryCompilationErrors.scala:802)
	at org.apache.spark.sql.catalyst.analysis.ResolveSessionCatalog$DatabaseNameI

In [5]:
spark.sparkContext.master

'spark://spark-master:7077'

In [6]:
spark.sparkContext.listJars()

AttributeError: 'SparkContext' object has no attribute 'listJars'